In [ ]:
import sys
sys.path.append('..')

In [ ]:
import torch
import esm

import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from DomainPrediction import BaseProtein
from DomainPrediction.utils import helper
from DomainPrediction.utils.constants import *

In [ ]:
model_path = '/data/users/kgeorge/workspace/esm2/checkpoints/esm2_t33_650M_UR50D.pt'
# model_path = '/data/users/kgeorge/workspace/esm2/checkpoints/esm2_t6_8M_UR50D.pt'
model, alphabet = esm.pretrained.load_model_and_alphabet(model_path)
batch_converter = alphabet.get_batch_converter()
model.eval()

In [ ]:
protein = BaseProtein(file='../../Data/6mfw_conformations/hm_6mfz_ATC.pdb')

In [ ]:
protein.get_residues(T_6mfw) ## T domain

In [ ]:
masked_query = ''.join(['<mask>' if i in T_6mfw else protein.sequence[i] for i in range(len(protein.sequence))])

In [ ]:
masked_query

In [ ]:
data = [
    ("protein1", masked_query)
]
batch_labels, batch_strs, batch_tokens = batch_converter(data)
batch_lens = (batch_tokens != alphabet.padding_idx).sum(1)

In [ ]:
batch_tokens.shape

In [ ]:
batch_tokens

In [ ]:
with torch.no_grad():
    results = model(batch_tokens, repr_layers=[33], return_contacts=True)
token_representations = results["representations"][33]

In [ ]:
results['logits'].shape, token_representations.shape

In [ ]:
results['logits']

In [ ]:
prob = torch.nn.functional.softmax(results['logits'], dim=-1)

In [ ]:
i = 596
plt.plot(prob[0, i, :])
plt.yscale('log')
torch.argmax(prob[0, i, :])

In [ ]:
inv = {v:k for k,v in alphabet.tok_to_idx.items()}

In [ ]:
torch.multinomial(prob[:, T_6mfw, :][0], 1)

In [ ]:
torch.argmax(prob[0, T_6mfw, :], dim=-1).numpy()

In [ ]:
''.join([inv[i] for i in torch.argmax(prob[:,1:-1,:][0, T, :], dim=-1).numpy()])

In [ ]:
protein.get_residues(T)